<a href="https://colab.research.google.com/github/HpDiniz/Analise-Financeira/blob/main/Projeto_de_Pesquisa_Mestrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings; 
warnings.simplefilter('ignore')

# 0. Install and Import Dependencies

In [ ]:
!pip install pystan
!pip install statsmodels
!pip install yahoo-finance
!pip install yfinance
!pip install xgboost
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 5.0 MB/s 
  Created wheel for yahoo-finance: filename=yahoo_finance-1.4.0-py3-none-any.whl size=7233 sha256=d85c778407ce8ca7646f211a3d248fa7f946a41623a26225de4568be8787cb23
  Stored in directory: /root/.cache/pip/wheels/74/3c/6e/504622cec9991612b9e3945fe8b375fef727854b42eb333c91
Successfully built yahoo-finance
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.4 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfu

In [ ]:
import re
import bs4
import json
import pickle
import requests
import datetime
import dateutil

from datetime import date
from bs4 import BeautifulSoup
from xgboost import XGBRegressor
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX

import numpy as np
import pandas as pd
import yfinance as yf

# 1. Read in Data and Process Dates

In [ ]:
first_day = pd.to_datetime('today').replace(day=1,hour=0,minute=0,second=0,microsecond=0)
this_month = (first_day).strftime("%Y-%m")
last_month = (first_day - relativedelta(months=1)).strftime("%Y-%m")

headers = {
    'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
}

In [ ]:
def updateColumn(columnName, line, newRow):
    if '<b>'+columnName+':</b>' in line:
        item = line.replace('<b>'+columnName+':</b>',"").replace("<li>","")
        item = item.replace("</li>","").replace("m<sup>2</sup>","")
        item = item.replace("N/A","").strip()
        newRow[columnName] = item

def converteData(datas, monthYearOnly):

    new_array = []
    meses = ["Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro"]

    for data in datas:

        item = data.split("/")
        mes = str(meses.index(item[0])+1)
        mes = ("0" + mes)[len(mes)-1:len(mes)+1]

        new_date = item[1] + "-" + mes

        if not monthYearOnly:
            new_date = new_date + "-01 00:00:00"
        
        new_array.append(new_date)
        
    return new_array

def xgboostPrediction(dataframe, column, month):

    new_df = dataframe.copy()

    predict_column = column + "_target"

    new_df[predict_column] = new_df[column].shift(-month)

    treino = new_df[:-1]
    validacao = new_df[-1:]

    if month > 1:
        treino = new_df[:-month]
        validacao = new_df[-month:-(month-1)]

    x_treino = treino.loc[:, [column]].values
    y_treino = treino.loc[:, [predict_column]].values

    modelo_xgb = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    modelo_xgb.fit(x_treino, y_treino)

    value = float(validacao[column][-1])
    predicao = modelo_xgb.predict(value)

    return (predicao[0])

def sarimaxPrediction(dataframe, columns, months):

    result_df = None

    for col in columns:
        arima_df_aux = dataframe[[col]]
        fit_arima = auto_arima(arima_df_aux, d=1, start_p=1, start_q=1, max_p=3, mar_q=3, seasonal=True, m=6, D=1, start_P=1,start_Q=1, max_P=2, max_Q=2, information_criterion='aic', trace=False, error_action='ignore', stepwise=True)

        model=SARIMAX(arima_df_aux, order=fit_arima.order, seasonal_order=fit_arima.seasonal_order)
        resultado_sarimax = model.fit()

        forecast = resultado_sarimax.get_forecast(steps=months)

        if result_df is None:
            result_df = pd.DataFrame.from_dict(forecast.predicted_mean)

        result_df[col] = forecast.predicted_mean

    return (result_df[columns])

def traditionalPredict(dfs, ativo, columns, months):

    dataframe = dfs[ativo]

    result_df = sarimaxPrediction(dataframe, columns, months)

    result_df['DividendYield'] = float(result_df['Dividends']/result_df['Close'])*100

    result_df['Ticker'] = ativo
    result_df['Prediction'] = True

    return result_df

def machineLearningPredict(dfs, ativo, columns, months_to_predict):

    xgboost_df = None

    dataframe = dfs[ativo]

    for m in range(len(months_to_predict)):

        predict_months = m+1

        result = {}

        for col in columns:
            result[col] = xgboostPrediction(dataframe,col, predict_months)

        result['DividendYield'] = float(result['Dividends']/result['Close'])*100

        result['Ticker'] = ativo
        result['Date'] = months_to_predict[m]
        result['Prediction'] = True

        result_df = pd.DataFrame.from_dict([result])

        result_df = result_df.set_index('Date')
        result_df.index = pd.to_datetime(result_df.index)

        if xgboost_df is None:
            xgboost_df = result_df.copy()
        else:
            xgboost_df = pd.concat([xgboost_df, result_df])

    return xgboost_df

In [ ]:
def get_funds():

    headers = {
        'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
            ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
    }

    url = 'https://www.fundsexplorer.com.br/ranking'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        df = pd.read_html(response.content, encoding='utf-8')[0]

    idx = df[df['Setor'].isna()].index
    df.drop(idx, inplace=True)

    df_funds = df.rename(columns={'Códigodo fundo': 'Ticker'})

    col_categorical = ['Ticker','Setor']
    df_funds[col_categorical] = df_funds[col_categorical].astype('category')

    df_funds.sort_values('Ticker', inplace=True)

    df_funds[col_categorical]

    return df_funds

def run_crawler():

    df_funds = get_funds()

    columns = {'Código': [],'Endereço': [], 'Bairro': [], 'Cidade': [], 'Área Bruta Locável': []}
    df_ativos = pd.DataFrame(columns)

    for fundo in dfs:

        response = requests.get(url_fundos + fundo, headers=headers)

        soup = bs4.BeautifulSoup(response.content, "html")
        div = soup.find("div", {"id": "dividends-chart-wrapper"})

        labels = re.findall('"labels":\[.*?\]', str(div))
        dividends = re.findall('"data":\[.*?\]', str(div))

        # parse data:
        dividends = json.loads("{" + dividends[0] + "}")['data']
        labels = json.loads("{" + labels[0] + "}")['labels']

        # converte "Março/2021" para "2021-03-01"
        datas = converteData(labels, False)

        for i in range(len(datas)):
            dfs[fundo].loc[dfs[fundo].index[dfs[fundo].index == datas[i]],'Dividends'] = dividends[i]
        
        #dfs[fundo]['Datetime'] = dfs[fundo].index
        #dfs[fundo]['DividendYield'] = float(dfs[fundo]['Dividends'][0]/dfs[fundo]['Close'][0]) * 100

        if ('Ativos do ') in str(response.content):

            print("Coletando ativos de " + fundo + "...")

            soup = BeautifulSoup(response.content,"lxml")
            w3schollsList = soup.find("div",id="fund-actives-items")

            lista = w3schollsList.find_all('ul')

            for l in lista:

                newRow = {}

                itemList = l.find_all('li')
                for it in itemList:
                    line = str(it)

                    for column in ['Endereço', 'Bairro', 'Cidade', 'Área Bruta Locável']:
                        updateColumn(column, line, newRow)

                if len(newRow) > 0:
                    newRow['Código'] = fundo
                    df_ativos = df_ativos.append(newRow, ignore_index = True)
        else:
            print(fundo + " não possui ativos.")

    columns_str = ['Código','Endereço', 'Bairro', 'Cidade']
    df_ativos[columns_str] = df_ativos[columns_str].astype("string")
    df_ativos['UF'] = df_ativos['Cidade'].str[-2:]
    df_ativos['Área Bruta Locável'] = pd.to_numeric(df_ativos['Área Bruta Locável'].str.replace(r"\.", "").str.replace(r"\,", "."), errors='coerce').astype(float)

    df_history = None
    for t in df['Códigodo fundo']:

        df_aux = dfs[t][['Datetime','Close','Dividends','DividendYield','Ticker','Prediction']]

        if df_history is None:
            df_history = df_aux.copy()
        else:
            df_history = df_history.append(df_aux, ignore_index = False)

    return df, df_ativos, df_history

def run_predictor(dfs, qnt_months_to_predict = 2):
    # Calculando datas de interesse
    months_to_predict = []

    for i in range(qnt_months_to_predict):
        add_month = dateutil.relativedelta.relativedelta(months=i)
        months_to_predict.append((first_day + add_month).strftime("%Y-%m-%d"))

    xgboost_dfs = None
    for t in dfs.keys():
        print("Predicting with Machine Learning: " + t + "...")
        aux_df = machineLearningPredict(dfs,t,['Close','Dividends'],months_to_predict)

        if xgboost_dfs is None:
            xgboost_dfs = aux_df.copy()
        else:
            xgboost_dfs = pd.concat([xgboost_dfs, aux_df])

    df_history = None
    xgboost_dfs['Datetime'] = xgboost_dfs.index

    for t in dfs.keys():

        dfs[t]['Datetime'] = dfs[t].index
        dfs[t]['DividendYield'] = float(dfs[t]['Dividends'][0]/dfs[t]['Close'][0]) * 100

        df_aux = dfs[t][['Datetime','Close','Dividends','DividendYield','Ticker','Prediction']]

        if df_history is None:
            df_history = df_aux.copy()
        else:
            df_history = df_history.append(df_aux, ignore_index = False)

        df_history = df_history.append(xgboost_dfs[(xgboost_dfs['Ticker'] == t)].copy(), ignore_index = False)

    return df_history

In [ ]:
df, df_ativos, df_history = run_crawler()

NameError: ignored

In [ ]:
df_history[df_history['Ticker'] == 'XTED11']

In [ ]:
df_history = run_predictor(dfs, 2)

In [ ]:
df_history[df_history['Ticker'] == 'HGLG11']

In [ ]:
dfs['XTED11']

# 2. Data Visualization

In [ ]:
import plotly.graph_objects as go

In [ ]:
df_ativos

In [ ]:
dfs['HGLG11'].head()

In [ ]:
fundo = 'HGLG11'

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(dfs[fundo].Date), y=list(dfs[fundo].High)))

# Set title
fig.update_layout(
    title_text="Preço do " + fundo + " ao longo dos anos"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [ ]:
dfs

In [ ]:
def get_info(html_content, id):

    soup = bs4.BeautifulSoup(html_content, "html")
    div = soup.find("div", {"id": id})

    labels = re.findall('"labels":\[.*?\]', str(div))
    dividends = re.findall('"data":\[.*?\]', str(div))

    dividends = json.loads("{" + dividends[0] + "}")['data']
    labels = json.loads("{" + labels[0] + "}")['labels']

    dates = converteData(labels, True)

    result = {}
    if len(dates) == len(dividends):
        for i in range(len(dates)):
            result[dates[i]] = dividends[i]

    return result

def get_adress(html_content, found):

    columns = {'Ticker': [],'Endereço': [], 'Bairro': [], 'Cidade': [], 'Área Bruta Locável': []}
    df_ativos = pd.DataFrame(columns)

    if ('Ativos do ') in str(html_content):

        soup = BeautifulSoup(html_content,"lxml")
        w3schollsList = soup.find("div",id="fund-actives-items")

        lista = w3schollsList.find_all('ul')

        for l in lista:

            newRow = {}

            itemList = l.find_all('li')
            for it in itemList:
                line = str(it)

                for column in ['Endereço', 'Bairro', 'Cidade', 'Área Bruta Locável']:
                    updateColumn(column, line, newRow)

            if len(newRow) > 0:
                newRow['Ticker'] = found
                df_ativos = df_ativos.append(newRow, ignore_index = True)
    #else:
        #print(found + " não possui ativos.")

    return df_ativos

def run_crawler(found):

    print("Coletando informações de " + found + "...")

    response = requests.get(url_fundos + found, headers=headers).content

    yields = get_info(response, 'yields-chart-wrapper')
    dividends = get_info(response, 'dividends-chart-wrapper')

    df_adress = get_adress(response, found)

    columns = {'Data': [], 'Ticker': [],'Fechamento': [], 'Dividendos': [], 'DividendYield': [], 'Predito': []}
    df_history = pd.DataFrame(columns)
    df_history['Predito'] = df_history['Predito'].astype('bool')

    for key in yields.keys():

        newRow = {}
        newRow['Data'] = key
        newRow['Ticker'] = found
        newRow['Fechamento'] = 100 * dividends[key]/yields[key]
        newRow['Dividendos'] = dividends[key]
        newRow['DividendYield'] = yields[key]
        newRow['Predito'] = False

        df_history = df_history.append(newRow, ignore_index = True)

    print(str(len(df_history)) + " dados de histórico e " + str(len(df_adress)) + " endereços foram encontrados.")

    return df_history, df_adress

In [ ]:
def get_all_funds():

    response = requests.get('https://www.fundsexplorer.com.br/ranking', headers=headers)
    if response.status_code == 200:
        df = pd.read_html(response.content, encoding='utf-8')[0]

    idx = df[df['Setor'].isna()].index
    df.drop(idx, inplace=True)

    df_funds = df.rename(columns={'Códigodo fundo': 'Ticker'})

    col_categorical = ['Ticker','Setor']
    df_funds[col_categorical] = df_funds[col_categorical].astype('category')

    df_funds.sort_values('Ticker', inplace=True)

    df_funds[col_categorical]

    return df_funds

def run_crawler():

    # Obtém a lista de todos os fundos imobiliários existentes
    df_funds = get_all_funds()

    df_funds = df_funds[df_funds['Ticker'] == 'ALZR11']

    # Percorre a lista de fundos para obter o histórico individual de cada um deles
    for fund in df_funds['Ticker']:

        print("Coletando informações de " + fund + "...")

        response = requests.get('https://www.fundsexplorer.com.br/funds/' + fund, headers=headers).content

        yields = get_info(response, 'yields-chart-wrapper')
        dividends = get_info(response, 'dividends-chart-wrapper')

        df_adress = get_adress(response, fund)

        columns = {'Data': [], 'Ticker': [],'Fechamento': [], 'Dividendos': [], 'DividendYield': [], 'Predito': []}
        df_history = pd.DataFrame(columns)
        df_history['Predito'] = df_history['Predito'].astype('bool')

        for key in yields.keys():

            newRow = {}
            newRow['Data'] = key
            newRow['Ticker'] = fund
            newRow['Dividendos'] = float(dividends[key])
            newRow['DividendYield'] = float(yields[key])
            newRow['Predito'] = False

            if float(dividends[key]) == 0.0 or float(yields[key]) == 0.0:
                print("Preciso descobrir o fechamento para a data: " + str(key))

                #ticker = yf.Ticker(fund + '.SA')
                #aux = ticker.history(interval='1d',period="max")

                #return aux, aux, aux
        
                newRow['Fechamento'] = 0.0
            else:
                newRow['Fechamento'] = 100 * dividends[key]/yields[key]

            df_history = df_history.append(newRow, ignore_index = True)

        print(str(len(df_history)) + " dados de histórico e " + str(len(df_adress)) + " endereços foram encontrados.")

    return df_funds, df_history, df_adress

In [ ]:
df_funds, df_history, df_adress = run_crawler()
df_history
#print(df_history)

Coletando informações de ALZR11...
Preciso descobrir o fechamento para a data: 2017-12
54 dados de histórico e 15 endereços foram encontrados.


,Data,Ticker,Fechamento,Dividendos,DividendYield,Predito
0,2017-12,ALZR11,0.00,0.000000,0.000000,False
1,2018-01,ALZR11,103.52,0.352054,0.340083,False
2,2018-02,ALZR11,102.80,0.780910,0.759640,False
3,2018-03,ALZR11,102.67,0.517545,0.504086,False
4,2018-04,ALZR11,99.36,0.642089,0.646225,False
5,2018-05,ALZR11,92.00,0.640027,0.695682,False
6,2018-06,ALZR11,92.51,0.640003,0.691821,False
7,2018-07,ALZR11,93.86,0.640300,0.682186,False
8,2018-08,ALZR11,92.07,0.641905,0.697192,False
9,2018-09,ALZR11,91.69,0.646603,0.705205,False


In [ ]:
ticker = yf.Ticker('ALZR11' + '.SA')
aux = ticker.history(interval='1d',period="max")
pd.set_option('display.max_rows', None)
aux['Close']

Date
2018-01-04     98.867981
2018-01-05     96.820061
2018-01-08     98.133080
2018-01-09     97.643150
2018-01-10     97.104218
2018-01-11     97.016045
2018-01-12     97.398178
2018-01-15     97.976295
2018-01-16     98.965958
2018-01-17     97.986099
2018-01-18     97.006241
2018-01-19     98.084091
2018-01-22     97.986099
2018-01-23     97.976295
2018-01-24     97.848915
2018-01-25     97.848915
2018-01-26     98.476028
2018-01-29     98.858177
2018-01-30     98.671997
2018-01-31     98.867981
2018-02-01     98.867981
2018-02-02     98.858177
2018-02-05     98.848373
2018-02-06     98.965958
2018-02-07     99.455887
2018-02-08     99.642067
2018-02-09    102.709030
2018-02-14    101.895744
2018-02-15    101.660576
2018-02-16    101.435204
2018-02-19    100.925682
2018-02-20    100.180992
2018-02-21    100.925682
2018-02-22    100.827698
2018-02-23     99.936020
2018-02-26    100.337769
2018-02-27     99.573479
2018-02-28     99.857635
2018-03-01     99.602867
2018-03-02    100.14

In [ ]:
fund = 'ABCP11'
response = requests.get('https://www.fundsexplorer.com.br/funds/' + fund, headers=headers).content

soup = bs4.BeautifulSoup(response, "html")
div = soup.find("div", {"id": "band-aid-chart"})

print(div)

<div data-chart="https://chart.fundsexplorer.com.br/chart/abcp11/" id="band-aid-chart">
<span class="loading">Carregando...</span>
</div>
